The iterator through the circuit iterates by moments, but within the moment, the operators are chosen arbitrarily. The previous op is not the operation before the current op on the same qubit.

In [ ]:
import cirq
import numpy as np

# The function copied from the source code, probe print statements added and generated by GPT-4
def apply_gate_cancellation(circuit: cirq.Circuit) -> cirq.Circuit:
    new_ops = []
    prev_op = None
    for i, op in enumerate(circuit.all_operations()):
        
        print(f"Step {i}: Current op = {op}")
        if prev_op:
            print(f"       Previous op = {prev_op}")
        else:
            print(f"       Previous op = None")

        if prev_op and isinstance(op.gate, cirq.EigenGate) and isinstance(prev_op.gate, cirq.EigenGate):
            if op.qubits == prev_op.qubits and type(op.gate) == type(prev_op.gate):
                exp_sum = op.gate.exponent + prev_op.gate.exponent
                print(f"       Same gate & qubits. Exponent sum = {exp_sum}")
                if np.isclose(exp_sum, 0.0, atol=1e-2):
                    print("       Gates cancel out! Skipping both.")
                    prev_op = None
                    continue
        if prev_op:
            new_ops.append(prev_op)
            print(f"       Adding previous op to new_ops")
        prev_op = op

    if prev_op:
        new_ops.append(prev_op)
        print(f"End: Adding last op {prev_op} to new_ops")

    return cirq.Circuit(new_ops)

# Create 2 qubits
q0, q1 = cirq.LineQubit.range(2)

# Build a circuit with canceling gates on different qubits
circuit = cirq.Circuit(
    cirq.X(q0)**0.5,
    cirq.X(q0)**-0.5,  # cancels with previous
    cirq.Z(q1),
    cirq.Y(q1)**0.3,
    cirq.Y(q1)**-0.3,  # cancels with previous
    cirq.X(q0)**0.25,
)

print("Original circuit:")
print(circuit)

new_circuit = apply_gate_cancellation(circuit)

print("\nOptimized circuit:")
print(new_circuit)


Original circuit:
0: ───X^0.5───X^-0.5───X^0.25───

1: ───Z───────Y^0.3────Y^-0.3───
<generator object AbstractCircuit.all_operations.<locals>.<genexpr> at 0x000001FA3FD04860>
Step 0: Current op = X**0.5(q(0))
       Previous op = None
Step 1: Current op = Z(q(1))
       Previous op = X**0.5(q(0))
       Adding previous op to new_ops
Step 2: Current op = X**-0.5(q(0))
       Previous op = Z(q(1))
       Adding previous op to new_ops
Step 3: Current op = Y**0.3(q(1))
       Previous op = X**-0.5(q(0))
       Adding previous op to new_ops
Step 4: Current op = Y**-0.3(q(1))
       Previous op = Y**0.3(q(1))
       Same gate & qubits. Exponent sum = 0.0
       Gates cancel out! Skipping both.
Step 5: Current op = X**0.25(q(0))
       Previous op = None
End: Adding last op X**0.25(q(0)) to new_ops

Optimized circuit:
0: ───X^0.5───X^-0.5───X^0.25───

1: ───Z─────────────────────────
